In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import html
from bs4 import BeautifulSoup
import requests
import time
import re
from selenium.webdriver.common.action_chains import ActionChains


In [11]:
def scrape_american_eagle(url, wait_time=1):
    try:
        # Set Chrome options for headless mode and define user agent
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
        chrome_options = Options()
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"user-agent={user_agent}")

        # Initialize the WebDriver with headless mode
        driver = webdriver.Chrome(options=chrome_options)
        
        # Open the webpage
        driver.get(url)

        # Save the source to extract item title
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_name_element = soup.find('h1', class_='cms-ae-product-name _product-name_yysft6')
        product_name = product_name_element.text.strip()

        # Wait the time before interacting with the page
        time.sleep(wait_time)

        # Scroll to a specific position on the page
        driver.execute_script("window.scrollTo(50, document.body.scrollHeight);")

        # Find the interactive element and move the mouse cursor to it
        interactive_element_xpath = '//*[@id="main-content-focus"]/div[2]/div[2]/div[2]/div/div[3]/div/div[1]/div[1]'
        interactive_element = driver.find_element(By.XPATH, interactive_element_xpath)
        actions = ActionChains(driver)
        actions.move_to_element(interactive_element).perform()

        # Click the interactive element
        interactive_element.click()

        # Wait for the loaded content to be visible
        loaded_content_xpath = '//*[@id="main-content-focus"]/div[2]/div[2]/div[2]/div/div[3]/div/div[1]/div[2]/div/div[2]'
        loaded_element = WebDriverWait(driver, wait_time).until(
            EC.visibility_of_element_located((By.XPATH, loaded_content_xpath))
        )

        # Once loaded, scrape the content
        dynamic_content = loaded_element.text

        # Make the item-material dictionary
        item = [(product_name, dynamic_content)]
        return process_item_data()
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
        
    finally:
        # Close the WebDriver
        driver.quit()

# Example usage:
url = "https://www.ae.com/us/en/p/women/t/classic-t/ae-hey-baby-ribbed-t/2370_9516_430?menu=cat4840004"

dynamic_content = scrape_american_eagle(url)
if dynamic_content:
    print(dynamic_content)

An error occurred: 'NoneType' object has no attribute 'text'


In [20]:
def process_item_data(item):
    processed_items = []
    print(item)
    for item, description in items:
        # Initialize dictionary with the item name
        item_dict = {'item': item}
        
        # Extract materials part before the care instructions
        materials_part = description.split("\n")[1]  # Assumes the second line contains materials info
        
        # Split materials by commas to separate each material component
        materials_list = materials_part.split(',')
        
        # Process each material component
        for material in materials_list:
            material = material.strip()
            if '%' in material:
                try:
                    # Split the material into percentage and name
                    percentage, material_name = material.split('%')
                    percentage = int(percentage.strip())
                    material_name = material_name.strip().lower()
                    
                    # Map to the dictionary
                    item_dict[material_name] = percentage
                except ValueError:
                    continue  # If conversion fails, skip this part

        # Append the processed dictionary to the result list
        processed_items.append(item_dict)

    return processed_items

processed_items = process_item_data(items)
for item in processed_items:
    print(item)


[('AE Luxe Ripped High V-Rise Jegging', 'Materials & Care\n89% Cotton, 8% Polyester, 3% Elastane\nColor may transfer when new...'), ('AE Next Level Ripped High-Waisted Jegging', 'Materials & Care\n61% Cotton, 21% Viscose, 10% Recycled Cotton, 1% Elastane\nColor may transfer when new...')]
{'item': 'AE Luxe Ripped High V-Rise Jegging', 'cotton': 89, 'polyester': 8, 'elastane': 3}
{'item': 'AE Next Level Ripped High-Waisted Jegging', 'cotton': 61, 'viscose': 21, 'recycled cotton': 10, 'elastane': 1}


# Below is the integrated code!

In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

def process_item_data(item):
    # Initialize the result list
    processed_items = []
    
    # Assuming 'item' is a list with a single tuple (product_name, description)
    for item_name, description in item:
        # Initialize dictionary with the item name
        item_dict = {'item': item_name}
        
        # Extract materials part before the care instructions
        materials_part = description.split("\n")[1]  # Assumes the second line contains materials info
        
        # Split materials by commas to separate each material component
        materials_list = materials_part.split(',')
        
        # Process each material component
        for material in materials_list:
            material = material.strip()
            if '%' in material:
                try:
                    # Split the material into percentage and name
                    percentage, material_name = material.split('%')
                    percentage = int(percentage.strip())
                    material_name = material_name.strip().lower()
                    
                    # Map to the dictionary
                    item_dict[material_name] = percentage
                except ValueError:
                    continue  # If conversion fails, skip this part

        # Append the processed dictionary to the result list
        processed_items.append(item_dict)

    return processed_items

def scrape_american_eagle(url, wait_time=1):
    try:
        # Set Chrome options for headless mode and define user agent
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
        chrome_options = Options()
        # chrome_options.add_argument("--headless")
        chrome_options.add_argument(f"user-agent={user_agent}")

        # Initialize the WebDriver with headless mode
        driver = webdriver.Chrome(options=chrome_options)
        
        # Open the webpage
        driver.get(url)

        # Save the source to extract item title
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_name_element = soup.find('h1', class_='cms-ae-product-name _product-name_yysft6')
        product_name = product_name_element.text.strip()

        # Wait the time before interacting with the page
        time.sleep(wait_time)

        # Scroll to 
        driver.execute_script("window.scrollTo(100, document.body.scrollHeight);")

        # Find the interactive element and move the mouse cursor to it
        interactive_element_xpath = '//*[@id="main-content-focus"]/div[2]/div[2]/div[2]/div/div[3]/div/div[1]/div[1]'
        interactive_element = driver.find_element(By.XPATH, interactive_element_xpath)
        actions = ActionChains(driver)
        actions.move_to_element(interactive_element).perform()

        # Click the interactive element
        interactive_element.click()

        # Wait for the loaded content to be visible
        loaded_content_xpath = '//*[@id="main-content-focus"]/div[2]/div[2]/div[2]/div/div[3]/div/div[1]/div[2]/div/div[2]'
        loaded_element = WebDriverWait(driver, wait_time).until(
            EC.visibility_of_element_located((By.XPATH, loaded_content_xpath))
        )

        # Once loaded, scrape the content
        dynamic_content = loaded_element.text

        # Make the item-material dictionary
        item = [(product_name, dynamic_content)]
        return process_item_data(item)
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
        
    finally:
        # Close the WebDriver
        driver.quit()

# Example usage:
urls = [
        "https://www.ae.com/us/en/p/women/t/classic-t/ae-hey-baby-ribbed-t/2370_9516_430?menu=cat4840004",
        "https://www.ae.com/us/en/p/women/cropped-jeans/ankle-jeans/ae-stretch-super-high-waisted-ripped-ankle-straight-jean/0435_4822_432?menu=cat4840004",
        "https://www.ae.com/us/en/p/women/high-waisted-shorts/high-waisted-short-shorts/ae-dreamy-drape-linen-blend-carpenter-short/4331_7848_309?menu=cat4840004",
        "https://www.ae.com/us/en/p/women/dresses/midi-maxi-dresses/ae-strapless-ruched-poplin-midi-dress/0395_7729_001?menu=cat4840004"
        ]

for url in urls:
    dynamic_content = scrape_american_eagle(url)
    if dynamic_content:
        print(dynamic_content)


[{'item': 'AE Hey Baby Ribbed T-Shirt', 'cotton': 57, 'recycled polyester': 38, 'elastane': 5}]
[{'item': 'AE Stretch Super High-Waisted Ripped Ankle Straight Jean', 'cotton': 89, 'recycled cotton': 10, 'elastane': 1}]
[{'item': 'AE Dreamy Drape Linen-Blend Carpenter Short', 'linen': 53, 'cotton': 24, 'viscose': 21, 'elastane': 2}]
[{'item': 'AE Strapless Ruched Poplin Midi Dress', 'cotton': 100}]
